# Exploratory Data Analysis (EDA) and Prediction on Bitcoin cryptocurrency

## Importing necessary libraries

---






In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
%matplotlib inline

# To hide warnings
import warnings
warnings.filterwarnings('ignore')

# Import for datareading:
import pandas_datareader as web
from pandas_datareader import data, wb
import requests
import datetime, dateutil.relativedelta
%pip install --upgrade mplfinance
import mplfinance as mpl

# Import tensorflow and keras:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
assert tf.__version__ >= "2.0"
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import  Dropout, Dense, LSTM
from tensorflow.keras.models import Sequential

# Initilise scaler:
scaler = MinMaxScaler()

     |████████████████████████████████| 71kB 4.9MB/s 


## Reading recent data from Yahoo Finance


---








In [3]:
# Bitcoin data from today till past 2 year ago:
end = datetime.datetime.now()
end_t = datetime.datetime.now().strftime('%Y-%m-%d')
start = datetime.datetime(end.year - 2,end.month,end.day).strftime('%Y-%m-%d')

In [4]:
# Bitcoin data in INR 
bitcoin = globals()['BTC-INR'] = web.DataReader('BTC-INR', 'yahoo', start, end_t).reset_index()

In [5]:
bitcoin.tail()


,Date,High,Low,Open,Close,Volume,Adj Close
570,2021-04-13,4874119.0,4619650.0,4775827.0,4734890.50,5.810924e+12,4734890.50
571,2021-04-14,4769128.5,4671974.0,4732302.0,4731307.50,4.554978e+12,4731307.50
572,2021-04-15,4750986.0,4480725.0,4727160.0,4591076.50,6.285173e+12,4591076.50
573,2021-04-16,4665594.0,4500174.0,4587880.0,4524215.50,4.930902e+12,4524215.50
574,2021-04-18,4546303.0,3977803.5,4212293.5,4130716.75,7.058531e+12,4130716.75


In [6]:
# Datatype
bitcoin.dtypes

Date         datetime64[ns]
High                float64
Low                 float64
Open                float64
Close               float64
Volume              float64
Adj Close           float64
dtype: object

In [7]:
bitcoin.shape

(575, 7)

## Exploratory Data Analysis (EDA) on bitcoin data

---







In [8]:
# Displaying first five rows
bitcoin.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2019-09-17,731490.6875,725478.5000,734975.8750,725961.0000,1.151007e+12,725961.0000
1,2019-09-18,735039.7500,703342.8125,726121.0000,732951.1875,1.423414e+12,732951.1875
2,2019-09-19,732415.8125,721472.4375,732944.3125,724993.9375,1.049163e+12,724993.9375
3,2019-09-20,725453.6875,712110.4375,725136.8750,713463.9375,9.559595e+11,713463.9375
4,2019-09-21,717307.0625,706490.9375,713777.1250,717018.5625,9.398239e+11,717018.5625


In [9]:
# Checking null values
bitcoin.isnull().sum().values

array([0, 0, 0, 0, 0, 0, 0])

In [10]:
# Checking unique values
bitcoin.nunique().values

array([573, 575, 575, 574, 575, 575, 575])

In [11]:
# Describing the columns
bitcoin.describe()


,High,Low,Open,Close,Volume,Adj Close
count,5.750000e+02,5.750000e+02,5.750000e+02,5.750000e+02,5.750000e+02,5.750000e+02
mean,1.309500e+06,1.239724e+06,1.273880e+06,1.280027e+06,2.676410e+12,1.280027e+06
std,1.177475e+06,1.100001e+06,1.137872e+06,1.145088e+06,1.664462e+12,1.145088e+06
min,3.973455e+05,3.099744e+05,3.717216e+05,3.700086e+05,8.640972e+11,3.700086e+05
25%,6.228961e+05,5.975309e+05,6.128964e+05,6.129255e+05,1.563123e+12,6.129255e+05
50%,7.350398e+05,7.123572e+05,7.272802e+05,7.274208e+05,2.345102e+12,7.274208e+05
75%,1.380204e+06,1.313590e+06,1.346009e+06,1.349281e+06,3.382939e+12,1.349281e+06
max,4.874119e+06,4.671974e+06,4.775827e+06,4.774301e+06,2.582998e+13,4.774301e+06


### Mean price of bitcoin in past 2 years


In [12]:
round(np.mean(bitcoin['Close']),2)

1280026.75

### Highest price of bitcoin in past 2 years

In [13]:
round(max(bitcoin['Close']),2)

4774301.0

### Lowest price of bitcoin in past 2 years

In [14]:
round(min(bitcoin['Close']),2)

370008.56

### Line graph for Closing price of bitcoin

In [15]:
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=bitcoin['Date']
                         , y=bitcoin['Adj Close']
                         , name='Bitcoin'
                        )
             , secondary_y=False
             )
# Add figure title
fig.update_layout(
    title_text='Bitcoin closing price'
)

# Set x-axis title
fig.update_xaxes(title_text='Date')

# Set y-axes titles
fig.update_yaxes(title_text='Bitcoin (INR)', secondary_y=False)

fig.show()

### Finding the moving average for a week, mid month and full month ie. 7,15 and 30 days

In [16]:
ma_day = [7,15,30]
for ma in ma_day:
    column_name = 'Moving avg %s' %(str(ma))
    bitcoin[column_name] = bitcoin['Adj Close'].rolling(window=ma).mean()

In [17]:
bitcoin.columns

Index(['Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close',
       'Moving avg 7', 'Moving avg 15', 'Moving avg 30'],
      dtype='object')

### 15 Day moving average graph 

In [18]:
fig = go.Figure()

# Add Traces for Adj close
fig.add_trace(go.Scatter(x=bitcoin['Date']
                         , y=bitcoin['Adj Close']
                         , name='Adj Close'
                         , visible=True
                        ))

# Add Traces for 15 days moving average
fig.add_trace(go.Scatter(x=bitcoin['Date']
                         , y=bitcoin['Moving avg 15']
                         , name='15 days moving average'
                         , visible=True
                        ))
# Set title
fig.update_layout(title_text='15 Days Moving Average')

# Set x and y-axis titles
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Price')

fig.show() 

### 30 days moving average graph

In [19]:
fig = go.Figure()

# Add Traces for adj close
fig.add_trace(go.Scatter(x=bitcoin['Date']
                         , y=bitcoin['Adj Close']
                         , name='Adj Close'
                         , visible=True
                        ))

# Add Traces for 30 days moving average
fig.add_trace(go.Scatter(x=bitcoin['Date']
                         , y=bitcoin['Moving avg 30']
                         , name='30 days moving average'
                         , visible=True
                        ))
# Set title
fig.update_layout(title_text='30 Days Moving Average')

# Set x and y-axis titles
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Price')

fig.show() 

### 7 days moving average graph

In [20]:
fig = go.Figure()

# Add Traces for adj close
fig.add_trace(go.Scatter(x=bitcoin['Date']
                         , y=bitcoin['Adj Close']
                         , name='Adj Close'
                         , visible=True
                        ))

# Add Traces for 7 days moving average
fig.add_trace(go.Scatter(x=bitcoin['Date']
                         , y=bitcoin['Moving avg 7']
                         , name='7 days moving average'
                        , visible=True
                        ))
# Set title
fig.update_layout(title_text='7 Days Moving Average')

# Set x and y-axis titles
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Price')

fig.show() 

### Candlestick chart for bitcoin analysis 

In [21]:
fig = go.Figure()

fig.add_trace(go.Candlestick(x=bitcoin['Date']
                             , open=bitcoin['Open']
                             , high=bitcoin['High']
                             , low=bitcoin['Low']
                             , close=bitcoin['Adj Close']
                            , visible = True
                             ,increasing_line_color= 'cyan', decreasing_line_color= 'gray'
                            ))
# Set title
fig.update_layout(title_text='Candlestick chart for bitcoin')

# Set x and y-axis titles
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Price')

# fig.show() 

### Difference between opening and closing price

In [22]:
# Difference between Close and Open:
bitcoin['Difference'] = bitcoin['Close'] - bitcoin['Open']
# Specifying colors for positive and negative difference
bitcoin['Color'] = np.where(bitcoin['Difference']<0, 'red', 'green')
bitcoin.head()

,Date,High,Low,Open,Close,Volume,Adj Close,Moving avg 7,Moving avg 15,Moving avg 30,Difference,Color
0,2019-09-17,731490.6875,725478.5000,734975.8750,725961.0000,1.151007e+12,725961.0000,NaN,NaN,NaN,-9014.8750,red
1,2019-09-18,735039.7500,703342.8125,726121.0000,732951.1875,1.423414e+12,732951.1875,NaN,NaN,NaN,6830.1875,green
2,2019-09-19,732415.8125,721472.4375,732944.3125,724993.9375,1.049163e+12,724993.9375,NaN,NaN,NaN,-7950.3750,red
3,2019-09-20,725453.6875,712110.4375,725136.8750,713463.9375,9.559595e+11,713463.9375,NaN,NaN,NaN,-11672.9375,red
4,2019-09-21,717307.0625,706490.9375,713777.1250,717018.5625,9.398239e+11,717018.5625,NaN,NaN,NaN,3241.4375,green


### Bar chart for price difference

In [23]:
fig = go.Figure()

# Add Trace for price difference
fig.add_trace(go.Bar(x=bitcoin['Date']
                     , y=bitcoin['Difference']
                     , marker_color=bitcoin['Color']
                     , visible = True
                    ))

# Set x and y-axis titles
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Price difference')

#fig.show()   

### Histogram for price 

In [34]:
fig = px.histogram(bitcoin, x="Adj Close")
fig.update_layout(title_text='Distributions of Adj Close', height=800)
fig.show()

In [24]:
# Colrelation
corr_matrix = bitcoin.corr()
corr_matrix

,High,Low,Open,Close,Volume,Adj Close,Moving avg 7,Moving avg 15,Moving avg 30,Difference
High,1.000000,0.998558,0.999187,0.999290,0.619995,0.999290,0.997669,0.993487,0.988159,0.110535
Low,0.998558,1.000000,0.998732,0.998998,0.599167,0.998998,0.996822,0.993229,0.989568,0.113277
Open,0.999187,0.998732,1.000000,0.998333,0.612268,0.998333,0.997976,0.993971,0.988916,0.080043
Close,0.999290,0.998998,0.998333,1.000000,0.610607,1.000000,0.996713,0.992459,0.987863,0.137442
Volume,0.619995,0.599167,0.612268,0.610607,1.000000,0.610607,0.613341,0.600716,0.577637,0.037945
Adj Close,0.999290,0.998998,0.998333,1.000000,0.610607,1.000000,0.996713,0.992459,0.987863,0.137442
Moving avg 7,0.997669,0.996822,0.997976,0.996713,0.613341,0.996713,1.000000,0.997632,0.992259,0.085905
Moving avg 15,0.993487,0.993229,0.993971,0.992459,0.600716,0.992459,0.997632,1.000000,0.996483,0.079444
Moving avg 30,0.988159,0.989568,0.988916,0.987863,0.577637,0.987863,0.992259,0.996483,1.000000,0.085581
Difference,0.110535,0.113277,0.080043,0.137442,0.037945,0.137442,0.085905,0.079444,0.085581,1.000000


### Line graph of Volume transaction distribution

In [25]:
fig = go.Figure()

# Add Traces for adj close
fig.add_trace(go.Scatter(x=bitcoin['Date']
                         , y=bitcoin['Volume']
                         , name='Volume of transactions'
                        , line=dict(color="#FF00FF")
                         , visible=True
                        ))

# Set title
fig.update_layout(title_text='Transaction volume')

# Set x and y-axis titles
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Volume')


## Price Prediction


---





### Data preprocessing

In [26]:
# Fill in missing values and sort index 
bitcoin_data = bitcoin[['Adj Close']].fillna(method='backfill').sort_index(ascending=True, axis=0)


In [27]:
# Scale and reshape Adj Close values
adj_price = bitcoin_data['Adj Close'].values.reshape(-1, 1)
bitcoin_data['Adj Close'] = scaler.fit_transform(adj_price)

### Splitting into training and testing

In [28]:
# Split data into training and testing 
prediction_days = 13

training = bitcoin_data[:len(bitcoin_data)-prediction_days]
testing = bitcoin_data[len(bitcoin_data)-prediction_days:]


### Training Data

In [29]:
# Define X_train, y_train:
training_set = training.values
training_set = scaler.fit_transform(training_set)

X_train = training_set[0:len(training_set)-1]
X_train = np.reshape(X_train, (len(X_train), 1, 1))

y_train = training_set[1:len(training_set)]

### LSTM recurrent neural network model 

In [30]:
# Sequential model
model = Sequential()
# Add the input layer and the LSTM layer
model.add(LSTM(units = 4, activation = 'sigmoid', input_shape=(None, 1)))
# Add the output layer
model.add(Dense(units = 1))

# Compile the model:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Using the training set to train the model:
history= model.fit(X_train, y_train, batch_size = 5, epochs = 100 )
history


Epoch 1/100
113/113 [==============================] - 1s 1ms/step - loss: 0.3951
Epoch 2/100
113/113 [==============================] - 0s 1ms/step - loss: 0.1597
Epoch 3/100
113/113 [==============================] - 0s 1ms/step - loss: 0.0948
Epoch 4/100
113/113 [==============================] - 0s 1ms/step - loss: 0.0878
Epoch 5/100
113/113 [==============================] - 0s 1ms/step - loss: 0.0580
Epoch 6/100
113/113 [==============================] - 0s 1ms/step - loss: 0.0575
Epoch 7/100
113/113 [==============================] - 0s 1ms/step - loss: 0.0505
Epoch 8/100
113/113 [==============================] - 0s 1ms/step - loss: 0.0609
Epoch 9/100
113/113 [==============================] - 0s 1ms/step - loss: 0.0511
Epoch 10/100
113/113 [==============================] - 0s 2ms/step - loss: 0.0441
Epoch 11/100
113/113 [==============================] - 0s 1ms/step - loss: 0.0580
Epoch 12/100
113/113 [==============================] - 0s 1ms/step - loss: 0.0397
Epoch 13/100


### Testing Data 

In [31]:
testing_set = testing.values

inputs = np.reshape(testing_set, (len(testing_set), 1))
inputs = scaler.transform(inputs)
inputs = np.reshape(inputs, (len(inputs), 1, 1))

predicted_price = model.predict(inputs)
predicted_price = scaler.inverse_transform(predicted_price)

###Line graph showing loss with epoch transition

In [32]:
loss_train = history.history['loss']
epochs = list(range(1,101))
fig = go.Figure()

# Add Traces
fig.add_trace(go.Scatter(x=epochs
                         , y=loss_train
                         , name='Training loss'
                         , line = dict(color='orange')
         
                        ))

# Set title
fig.update_layout(title_text='Loss with epoch transition')

# Set x and y-axis titles
fig.update_xaxes(title_text='Epoch')
fig.update_yaxes(title_text='Loss')

fig.show()   

## Line graph of Real and Predicted price


---







In [33]:
fig = go.Figure()

# Add Traces
fig.add_trace(go.Scatter(x=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
                         , y=testing_set[:, 0]
                         , name='Price real'
                         , line = dict(color='purple', width=2, dash='dash')
                         , visible=True
                        ))

fig.add_trace(go.Scatter(x=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
                         , y=predicted_price[:, 0]
                         , name='Price predicted'
                         , line = dict(color='gold', width=2, dash='dash')
                         , visible=True
                        ))
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
# Set title
fig.update_layout(title_text='Bitcoin Price Prediction')

# Set x and y-axis titles
fig.update_xaxes(title_text='Number of Days')
fig.update_yaxes(title_text='Price')

fig.show()   